<a href="https://colab.research.google.com/github/thekhan314/CovidBrazilMod3/blob/master/COVID_REDUX_last_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries and Functions


In [ ]:
import sklearn

In [ ]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [1]:
#from khantools import *
import pandas as pd
import numpy as np

import pprint as pp
import matplotlib.pyplot as plt
import seaborn as sns
import operator as operator

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize

from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

C:\Users\umark\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [ ]:
# Evaluate empty columns
def cols_with_data(dataframe,threshold,plot=False,axis=0):
    ''' returns a series with all columns that are filled to at least threshold percentage or higher'''
    
    counts = dataframe.count(axis=axis).sort_values()
    
    if axis == 0: 
        counts = counts/len(dataframe)
    else:
        counts = counts/len(dataframe.columns)
        
    non_zero_cols = counts[counts >= threshold ]
    
    if plot == True:
        fig,ax = plt.subplots()
        ax.barh(non_zero_cols.index,non_zero_cols)
        fig.set_size_inches(6,9)
        fig.show()
        print("Features left:{}".format(len(non_zero_cols)))
    return non_zero_cols

def value_counts (df,value, oper):
    ''' takes in a datframe of float values, returns count of zero values '''
    dict = {}
    
    ops = {'>': operator.gt,
           '<': operator.lt,
           '>=': operator.ge,
           '<=': operator.le,
           '=': operator.eq}
    
    for col in df.columns:
        matches = sum(ops[oper](df[col],value))
        
        dict[col] = matches
    
    df = pd.Series(dict)
    return df


Lets import the data and run a preliminary report

In [ ]:
df = pd.read_excel('dataset.xlsx')
report_prelim = report1(df,5)

It appears that the most commonly occuring values aside from the ones that are always there is what appears to be a battery of standard tests. I think it might be a good idea to discard all the rows where these tests were not performed, as they are much too sparse to be of any use to us. Lets keep only rows where standard tests were done, and see if that leaves us with enough data to make a meaningful model.

In [ ]:
df = df[df['Parainfluenza 2'].notnull()]
report_post_trim = report1(df,6)
print(len(df))
#display(report_post_trim)

# Data Cleanup and sorting

## Rename & Clean Columns

In [ ]:
# Lets change the names of some columns to something easier to work with
new_names = {
    'Patient addmited to intensive care unit (1=yes, 0=no)':'intensive',
    'Patient addmited to semi-intensive unit (1=yes, 0=no)':'semi_int',
    'Patient addmited to regular ward (1=yes, 0=no)':'reg_ward',
    'SARS-Cov-2 exam result':'sars_cov2',
    'Patient age quantile':'age_quant',
    'Patient ID':'patient_id'
}

df = df.rename(columns=new_names)
df.set_index(keys='patient_id',inplace=True)

# convert age_quant to float

df['age_quant'] = df['age_quant'].astype(float)
#delete empty columns

empty_cols = list(report_post_trim[report_post_trim['% empty'] == 100.0].index)
df = df.drop(columns=empty_cols)



## Reduce features

In [ ]:
# all data: filled values in each column 
col_fill = cols_with_data(df,threshold=0.03)

meaty_cols = list(col_fill[col_fill > 0.1].index)
meaty_cols.append('Lactic Dehydrogenase')

df = df[meaty_cols]

## Subset features

In [ ]:
df_types = df.dtypes

float_features = list(df_types[df_types == 'float64'].index)
obj_features = list(df_types[df_types == 'object'].index)

high_imp= ['Lactic Dehydrogenase','Lymphocytes','Proteina C reativa mg/dL']
admission_features = ['intensive','semi_int','reg_ward']



for col in admission_features:
    df[col] = df[col].astype(float)

## Convert Categoricals

In [ ]:
report2 = report1(df[obj_features],3).sort_values('unique values')
display(report2)

Lets also drop all the unary features because it seems they are all basically negative and dont seem to add much information.

Lets convert the binary categoricals to floats. 

For tests that have a 'not done' value, I will treat 'not done' as negative, since its safe to assume if t he test wasnt done the doctor was expecting a negative anyways. 

Urine Aspect color and crystals I will drop. They dont sound too important, and dont comprise alot of rows.I will also drop standard tests, as I feel they will only add noise.  

Urine PH and Urine Leukocytes should be converted to float



In [ ]:
binary_conversions = list(report2[report2['unique values'] == 2].index)

for col in binary_conversions:
    df[col] = df[col].replace(['not_detected','negative','absent','normal'],0)
    df[col] = df[col].replace(['detected','positive','not_done'],1)
    df[col] = df[col].astype(float)

df['sars_cov2'] = df['sars_cov2'].replace(['negative'],0)
df['sars_cov2'] = df['sars_cov2'].replace(['positive'],1)
df.drop('Parainfluenza 2',axis=1,inplace=True)

report3 = report1(df,6)

display(report3)

## Fill Empties

In [ ]:
full_cols = cols_with_data(df,1)
full_feats = list(full_cols.index)

unfull_feats = list(df.drop(full_feats,axis=1).columns)


In [ ]:
filled_feats = full_feats.copy()


for feature in unfull_feats:
    df_vals_present = df.loc[df[feature].notnull()]
    
    X = df_vals_present[filled_feats]
    y = df_vals_present[feature]
    
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    if feature in float_features:
        pipe = Ridge(alpha=0.5)
       
        
    elif feature in obj_features:
        
        pipe = LogisticRegression(C=1e5, solver='lbfgs',class_weight='balanced',max_iter=10000)
        
    
    pipe.fit(X,y)
    
    df_predictors = df.loc[df[feature].isna()][filled_feats]
    y_hat = pipe.predict(df_predictors)
    
    df.loc[df[feature].isna(),feature] = y_hat
    
    filled_feats.append(feature)

In [ ]:
display(report1(df,4))

## Unified target column

In [ ]:
for feat in admission_features:
    df.loc[df[feat] == 1,'adm_status'] = feat

df['adm_status'].fillna(value='not_adm',inplace=True)
df = df.drop(columns=admission_features)

# Train Classifiers

In [ ]:
smote = SMOTE()

X = df.drop('adm_status',axis=1)
scaler = StandardScaler()
X = scaler.fit_transform(X)

y = df['adm_status']

X_train, X_test, y_train, y_test = train_test_split(X,y,  test_size=0.25, random_state=42)

X_train_smote, y_train_smote = smote.fit_sample(X_train, y_train) 

## Baseline Model (Logistic Regression)

In [ ]:
clf = LogisticRegression()

params = {
    'C': [1e3,1e5,1e10,1e20],
    'solver':['lbfgs', 'liblinear'],
    'max_iter':[10,100],
}

clf = LogisticRegression()

gs_base = GridSearchCV(
    estimator=clf,
    param_grid=params,
    cv=5,
    n_jobs = -1,
    verbose = 1,
)

gs_base.fit(X_train_smote,y_train_smote)

base_clf = gs_base.best_estimator_

base_clf.fit(X_train_smote,y_train_smote)

base_cf_plot = plot_confusion_matrix(base_clfclf,X_test,y_test,ax=ax,normalize='true')

## binarized

In [ ]:

smote = SMOTE()

X = df.drop('adm_status',axis=1)
scaler = StandardScaler()
X = scaler.fit_transform(X)
label_list = df['adm_status'].value_counts().index.to_list()
y = label_binarize(df['adm_status'],label_list)

X_train, X_test, y_train, y_test = train_test_split(X,y,  test_size=0.25, random_state=42)

X_train_resampled, y_train_resampled = smote.fit_sample(X_train, y_train) 

In [ ]:
estimator = LogisticRegression(C=1000, solver='liblinear',multi_class='ovr',max_iter=100)

clf = OneVsRestClassifier(estimator)

clf.fit(X_train_resampled,y_train_resampled)

y_score = clf.decision_function(X_test)
y_pred = clf.predict(X_test)
n_classes = y.shape[1]

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

fig,ax = plt.subplots(len(label_list),figsize=(20,40))

for i in range(0,len(label_list)):
    ax[i].plot(fpr[i], tpr[i], color='darkorange',
            lw=2, label='ROC curve (area = %0.2f)' % roc_auc[2])
    ax[i].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    ax[i].set_xlim([0.0, 1.0])
    ax[i].set_ylim([0.0, 1.05])
    ax[i].set_xlabel('False Positive Rate')
    ax[i].set_ylabel('True Positive Rate')
    ax[i].set_title(label_list[i])
    ax[i].legend(loc="lower right")
    
    print(label_list[i], classification_report(y_test[:,i],y_pred[:,i]), auc(fpr[i],tpr[i]))
    

In [ ]:
from sklearn.metrics import plot_confusion_matrix
class_labels = y_train.value_counts().index.to_list()
fig,ax = plt.subplots(figsize=(20,10))

cf_plot = plot_confusion_matrix(clf,X_test,y_test,ax=ax,normalize='true')



# Relabel and Retry

In [ ]:
df2 = df.copy()

semi_indexes = df2.loc[df2['adm_status'] == 'semi_int',:].index.to_list()

df_train = df2.drop(semi_indexes,axis=0)

X = df_train.drop('adm_status',axis=1)
y = df_train['adm_status']

X_test = df2.loc[semi_indexes,:].drop('adm_status',axis=1)



In [ ]:
len(semi_indexes)

In [ ]:
weight_dict = {
    'not_adm': 1,
    'reg_ward': 10,
    'intensive': 70
}

clf = LogisticRegression(C=1000,max_iter=100,solver='liblinear',class_weight=weight_dict)

clf.fit(X,y)

y_predict = clf.predict(X_test)

df2.loc[semi_indexes,'adm_status'] = y_predict

In [ ]:
df2['adm_status'].value_counts()

In [ ]:
df['adm_status'].value_counts()

# retry

In [ ]:
smote = SMOTE()

X = df2.drop('adm_status',axis=1)
scaler = StandardScaler()
X = scaler.fit_transform(X)

y = df2['adm_status']

X_train, X_test, y_train, y_test = train_test_split(X,y,  test_size=0.25, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_sample(X_train, y_train) 

In [ ]:
params = {
    'C': [1e3,1e5,1e10,1e20],
    'solver':['lbfgs', 'liblinear'],
    'max_iter':[10,100,1000],
}

scorer = make_scorer(recall_score, average = 'weighted')
gs_logistic = GridSearchCV(
    estimator=clf,
    param_grid=params,
    cv=5,
    n_jobs = -1,
    verbose = 1,
    scoring = scorer
)
gs_logistic.fit(X_train_resampled,y_train_resampled)

gs_logistic.best_score_


In [ ]:
gs_logistic.best_score_

In [ ]:
clf = gs_logistic.best_estimator_

clf.fit(X_train_resampled,y_train_resampled)

y_pred = clf.predict(X_test)

class_report_1 = classification_report(y_test,y_pred)

print(class_report_1)

# Relabel 2

In [ ]:
print(y.value_counts())

In [ ]:
weights_dict = [{
    'not_adm':1,
    'reg_ward':20,
    'intensive':60
},{
    'not_adm':1,
    'reg_ward':15,
    'intensive':50
},{
    'not_adm':1,
    'reg_ward':30,
    'intensive':60
}
]

In [ ]:
clf = LogisticRegression()


params = {
    'C': [1e3,1e5,1e10,1e20],
    'solver':['lbfgs', 'liblinear'],
    'max_iter':[10,100,1000],
    'class_weight':weights_dict
}

scorer = make_scorer(recall_score, average = 'weighted')
gs_logistic = GridSearchCV(
    estimator=clf,
    param_grid=params,
    cv=5,
    n_jobs = -1,
    verbose = 1,
    scoring = scorer
)
gs_logistic.fit(X_train_resampled,y_train_resampled)

gs_logistic.best_score_
gs_logistic.best_params_

In [ ]:
gs_logistic.best_score_

In [ ]:
clf = gs_logistic.best_estimator_

clf.fit(X_train_resampled,y_train_resampled)

y_pred = clf.predict(X_test)

class_report_1 = classification_report(y_test,y_pred)

print(class_report_1)

In [ ]:
weights_dict = [{
    'not_adm':1,
    'reg_ward':10,
    'semi_int':100,
    'intensive':200
},{
    'not_adm':1,
    'reg_ward':25,
    'semi_int':200,
    'intensive':500
},{
    'not_adm':1,
    'reg_ward':50,
    'semi_int':400,
    'intensive':750
}
]

## Logistic Regresison

## K Nearest Neighbors

In [ ]:
smote = SMOTE()
df = df2.copy()
X = df.drop('adm_status',axis=1)
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = df['adm_status']

X_train, X_test, y_train, y_test = train_test_split(X,y,  test_size=0.25, random_state=42)

X_train_resampled, y_train_resampled = smote.fit_sample(X_train, y_train) 


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()

clf.fit(X_train_resampled,y_train_resampled)

y_pred = clf.predict(X_test)

print(classification_report(y_test,y_pred))

In [ ]:
clf = KNeighborsClassifier()

params = {
    'n_neighbors': [i for i in range(10,20)],
    'weights':['uniform','distance'],
    'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size':[5,10,15,20,25,30,40],
    'p':[i for i in range(1,4)]
}

scorer = make_scorer(recall_score, average = 'weighted')

gs_kneighb_class = GridSearchCV(
    estimator=clf,
    param_grid=params,
    cv=5,
    n_jobs = -1,
    verbose = 1,
    scoring = scorer
)

gs_kneighb_class.fit(X_train,y_train)

gs_kneighb_class.best_score_
gs_kneighb_class.best_params_

In [ ]:
clf = gs_kneighb_class.best_estimator_

clf.fit(X_train_resampled,y_train_resampled)

y_pred = clf.predict(X_test)

class_report_1 = classification_report(y_test,y_pred)

print(class_report_1)

# Decision Tree

In [ ]:
clf = DecisionTreeClassifier()

params = {
    'criterion': ['entropy','gini'],
    'splitter':['best', 'random'],
    'max_depth':[i for i in np.linspace(1, 32, 8, endpoint=True)],
    'min_samples_split':[i for i in np.linspace(0.1, 1.0, 10, endpoint=True)],
    'min_samples_leaf':[i for i in np.linspace(0.1, 0.5, 5, endpoint=True)]
}

scorer = make_scorer(recall_score, average = 'weighted')

gs_dec_tree = GridSearchCV(
    estimator=clf,
    param_grid=params,
    cv=5,
    n_jobs = -1,
    verbose = 1,
    scoring = scorer
)
gs_dec_tree.fit(X_train,y_train)

gs_dec_tree.best_score_
gs_dec_tree.best_params_

In [ ]:
clf = gs_dec_tree.best_estimator_

clf.fit(X_train_resampled,y_train_resampled)

y_pred = clf.predict(X_test)

class_report_1 = classification_report(y_test,y_pred)

print(class_report_1)

In [ ]:
gs_dec_tree.best_score_

In [ ]:
tree_dict = {'criterion': 'entropy',
 'max_depth': 1.0,
 'min_samples_leaf': 0.1,
 'min_samples_split': 0.1,
 'splitter': 'best'}

tree_clf = DecisionTreeClassifier(criterion='entropy',
 max_depth=1.0,
 min_samples_leaf= 0.1,
 min_samples_split= 0.1,
 splitter= 'best')

tree_clf.fit(X_train_resampled,y_train_resampled)
y_score = tree_clf.predict(X_test)

class_report_1 = classification_report(y_test,y_score)

print(class_report_1)


## Random Forest

In [ ]:

clf = RandomForestClassifier()
clf.fit(X_train_resampled,y_train_resampled)
 
y_score = clf.predict(X_test)

class_report_1 = classification_report(y_test,y_score)

print(class_report_1)

## XGboost

In [ ]:
import xgboost as xgb
clf = xgb.XGBClassifier( objective='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10,scale_pos_weight=15)
clf.fit(X_train_resampled,y_train_resampled)

y_score = clf.predict(X_test)

class_report_1 = classification_report(y_test,y_score)

print(class_report_1)

## Support Vector Machines

In [ ]:
clf = SVC()

params = {
    'C':[1,100,1000],
    'kernel':['linear','poly','rbf','sigmoid'],
    'degree': [3],
    'gamma':['scale'],
    'class_weight':[None,'balanced'],
}

scorer = make_scorer(recall_score, labels=['intensive','reg_ward'],average = 'macro')
gs_svc = GridSearchCV(
    estimator=clf,
    param_grid=params,
    cv=5,
    n_jobs = -1,
    verbose = 1,
    scoring = scorer
)
gs_svc.fit(X_train_resampled,y_train_resampled)

gs_svc.best_score_
gs_svc.best_params_

In [ ]:

clf = gs_svc.best_estimator_

clf.fit(X_train_resampled,y_train_resampled)

y_pred = clf.predict(X_test)

class_report_1 = classification_report(y_test,y_pred,output_dict=True)
report = pd.DataFrame(class_report_1)
print(report)

In [ ]:
gs_svc.best_score_

In [ ]:
gs_svc.cv_results_

In [ ]:
score = recall_score(y_test,y_pred,labels=['intensive'],average = None)
print(score)

In [ ]:
clf = SVC()

params = {
    'C':[1,100,1000],
    'kernel':['linear','poly','rbf','sigmoid'],
    'degree': [3],
    'gamma':['scale'],
    'class_weight':[None,'balanced'],
}

scorer = make_scorer(recall_score, labels=['intensive'],average = 'macro')
gs_svc = GridSearchCV(
    estimator=clf,
    param_grid=params,
    cv=5,
    n_jobs = -1,
    verbose = 1,
    scoring = scorer
)
gs_svc.fit(X_train_resampled,y_train_resampled)

gs_svc.best_score_
gs_svc.best_params_


clf = gs_svc.best_estimator_

clf.fit(X_train_resampled,y_train_resampled)

y_pred = clf.predict(X_test)

class_report_1 = classification_report(y_test,y_pred,output_dict=True)
report = pd.DataFrame(class_report_1)
print(report)

In [ ]:
gs_svc.best_score_

In [ ]:
clf = SVC()

params = {
    'C':[1,100,1000],
    'kernel':['linear','poly','rbf','sigmoid'],
    'degree': [3],
    'gamma':['scale'],
    'class_weight':[None,'balanced'],
}

scorer = make_scorer(recall_score, labels=['intensive','reg_ward'],average = 'weighted')
gs_svc = GridSearchCV(
    estimator=clf,
    param_grid=params,
    cv=5,
    n_jobs = -1,
    verbose = 1,
    scoring = scorer
)
gs_svc.fit(X_train_resampled,y_train_resampled)

gs_svc.best_score_
gs_svc.best_params_


clf = gs_svc.best_estimator_

clf.fit(X_train_resampled,y_train_resampled)

y_pred = clf.predict(X_test)

class_report_1 = classification_report(y_test,y_pred,output_dict=True)
report = pd.DataFrame(class_report_1)
print(report)

In [ ]:
y_pred_prob = clf.predict_proba(X_test)

precision, recall, thresholds = precision_recall_curve(y_test,y_pred_[rob])

In [ ]:
smote = SMOTE()

X = df2.drop('adm_status',axis=1)
scaler = StandardScaler()
X = scaler.fit_transform(X)
label_list = df2['adm_status'].value_counts().index.to_list()
y = label_binarize(df2['adm_status'],label_list)

X_train, X_test, y_train, y_test = train_test_split(X,y,  test_size=0.25, random_state=42)

X_train_resampled, y_train_resampled = smote.fit_sample(X_train, y_train) 

In [ ]:
from sklearn.metrics import precision_recall_curve
estimator = clf
ovr = OneVsRestClassifier(estimator)

ovr.fit(X_train_resampled,y_train_resampled)

y_score = ovr.decision_function(X_test)
y_pred = ovr.predict(X_test)
n_classes = y.shape[1]

precision = dict()
recall = dict()

for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i], y_score[:, i])

fig,ax = plt.subplots(len(label_list),figsize=(20,40))

for i in range(0,len(label_list)):
    ax[i].plot(recall[i],precision[i], color='darkorange',
            lw=2)
    
    ax[i].set_xlim([0.0, 1.0])
    ax[i].set_ylim([0.0, 1.05])
    ax[i].set_xlabel('recall')
    ax[i].set_ylabel('precision')
    ax[i].set_title(label_list[i])
    ax[i].legend(loc="lower right")
    
    